In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
pd.options.display.max_columns = 999

grade_scale = {'A+': 13, 'A': 12, 'A-': 11, 'B+': 10, 'B': 9, 'B-': 8, 'C+': 7, 'C': 6, 'C-': 5, 'D+': 4, 'D': 3, 'D-': 2, 'F': 1, 'NA': 0}

courses = {"0":1, "1":2, "3":2, "3+": 4}
courses_imp = {'Most important': 4, 'Second-most important': 3, 'Third-most important': 2, 'Fourth-most important': 1}
courses_int = {'Most interesting': 4, 'Second-most interesting': 3, 'Third-most interesting': 2, 'Fourth-most interesting': 1}

scale_map = {"courses": courses, "courses_imp": courses_imp, "courses_int": courses_int}

In [2]:
df = pd.read_csv("../data/class_data.csv")
df_interested = df[df["pre_studyinterest"] == "Yes"]
df_var_sel = pd.read_excel("../data/student_interested_var_decision.xlsx", sheet_name = None)
df_codebook = pd.read_excel("../data/codebook.xlsx", sheet_name = None)

In [3]:
df_interested.head()

,randomid,intervention,asia,roster6_21,roster6_29,roster7_2,courseformat,coursecode,departmentnamecode,coursenum,coursetitle,section,acaterm,year,acayear,gr_lec1,gr_lec2,gr_lec3,gr_lec4,gr_lec5,gr_lec6,gr_lec7,gr_lec8,gr_lec9,gr_lec10,gr_lec11,gr_lec12,gr_lec16,gr_lec13,gr_lec14,gr_lec15,gr_lec18,gr_lec17,gr_readq1,gr_readq2,gr_readq3,gr_readq4,gr_readq5,gr_readq6,gr_readq7,gr_readq8,gr_readq9,gr_readq10,gr_readq11,gr_readq12,gr_readq13,gr_readq14,gr_readq15,gr_readq16,gr_readq17,gr_readq18,gr_revq1,gr_revq2,gr_revq3,gr_revq4,gr_revq5,gr_survey1,gr_survey2,gr_survey3,gr_survey4,gr_survey5,gr_exam1,gr_exam2,gr_exam3,gr_particf,gr_readqf,gr_revqf,gr_surveyf,gr_examf,gr_finalc,gr_finalf,gr_gradec,gr_gradef,gr_status,pre_start,pre_end,pre_progress,pre_duration,pre_finished,externalreference,pre_latitude,pre_longitude,pre_distribution,userlanguage,pre_studyinterest,pre_giftcard,pre_reas,pre_reas_o,pre_coursegoals,pre_coursegoals_o,pre_studydays,pre_syllabus,pre_studyplan,pre_studyplan_o,pre_study,pre_wgrade,pre_egrade,pre_difgrade,pre_difgrade_o,pre_badgrade,pre_courses,pre_courses_imp,pre_courses_int,pre_oact1,pre_oact2,pre_oact3,pre_oact4,pre_oact5,pre_oact6,pre_oact7,pre_oact8,pre_oact9,pre_oact10,pre_courserank,pre_oact1rank,pre_oact2rank,pre_oact3rank,pre_oact4rank,pre_oact5rank,pre_oact6rank,pre_oact7rank,pre_oact8rank,pre_oact9rank,pre_oact10rank,pre_oact1hrs,pre_oact2hrs,pre_oact3hrs,pre_oact4hrs,pre_oact5hrs,pre_oact6hrs,pre_oact7hrs,pre_oact8hrs,pre_oact9hrs,pre_oact10hrs,pre_orsh1,pre_orsh2,pre_orsh3,pre_orsh4,pre_orsh5,pre_orsh6,pre_orsh7,pre_er1,pre_er3,pre_er4,pre_er2,pre_er7x,pre_er8x,pre_olsrl1,pre_olsrl2,pre_olsrl3,pre_olsrl4,pre_olsrl5,pre_onlexp1,pre_onlexp2,pre_abil_genbio1,pre_abil_genbio3,pre_abil_genchem1,pre_abil_genchem3,pre_se1,pre_se2,pre_se3,pre_se4,pre_se5,pre_util1,pre_util2,pre_util3,pre_int1,pre_int2,pre_int3,pre_int4,pre_util4,pre_util5,pre_att1,pre_att2,pre_att3,pre_cost1,pre_cost4,pre_cost7,pre_cost2,pre_cost5,pre_cost8,pre_cost3,pre_cost6,pre_cost9,pre_iemot1,pre_iemot2,pre_iemot3,pre_iemot4,pre_iemot5,pre_iemot6,pre_iemot7,pre_agqmap1,pre_agqmap2,pre_agqmap3,pre_agqmap5,pre_agqmap6,pre_agqmap7,pre_agqmap10,pre_agqmap12,pre_agqmap16,pre_agqmap17,pre_eng1st,pre_begscl,pre_parents,pre_onlcrd,pre_resid,pre_major,pre_cong,pre_persist,pre_extraquestion,pre_essayquestion,rosterid,pre_status,post_start,post_end,post_progress,post_duration,post_finished,post_latitude,post_longitude,post_distribution,post_studydays,post_study,post_studyplan_chg,post_studyplan_chgw,post_studyplan_chgh,post_wgrade,post_egrade,post_badgrade,post_wgradef,post_egradef,post_badgradef,post_egrade_let,post_courses,post_courses_imp,post_courses_int,post_oactcompx1,post_oactcompx2,post_oactcompx3,post_oactcompx4,post_oactcompx5,post_oactcompx6,post_oactcompx7,post_oactcompx8,post_oactcompx9,post_oactcompx10,post_oact11,post_oact12,post_oact13,post_oact14,post_oact15,post_oact1hrs,post_oact2hrs,post_oact3hrs,post_oact4hrs,post_oact5hrs,post_oact6hrs,post_oact7hrs,post_oact8hrs,post_oact9hrs,post_oact10hrs,post_oact11hrs,post_oact12hrs,post_oact13hrs,post_oact14hrs,post_oact15hrs,post_courserank,post_oact1rank,post_oact2rank,post_oact3rank,post_oact4rank,post_oact5rank,post_oact6rank,post_oact7rank,post_oact8rank,post_oact9rank,post_oact10rank,post_oact11rank,post_oact12rank,post_oact13rank,post_oact14rank,post_oact15rank,post_activ1,post_activ2,post_activ3,post_activ4,post_activ5,post_activ6,post_activ7,post_aca1,post_aca2,post_aca3,post_aca4,post_aca5,post_orsh1,post_orsh2,post_orsh4,post_orsh5,post_orsh6,post_orsh7,post_er1,post_er3,post_er4,post_er2,post_er7x,post_er8x,post_olsrl1,post_olsrl2,post_olsrl3,post_olsrl4,post_olsrl5,post_canv13,post_canv4,post_canv2,post_canv1,post_canv4a,post_canv5,post_crs4,post_canv12,post_canv6,post_canv26,post_crs19,post_goalrmb,post_goalhelpd,post_goalhelpw,post_abil_genbio1,post_abil_genbio3,post_abil_genchem1,post_abil_genchem3,post_se1,post_se2,post_se3,post_se4,post_se5,post_util1,post_util2

In [4]:
pre_post_vars = df_var_sel["Pre & Post Keep"]["Var"].tolist()
pre_post_dict = defaultdict(list)
for c, name_stem in zip(df_var_sel["Pre & Post Keep"]["Var"], df_var_sel["Pre & Post Keep"]["Name Stem"]):
    pre_post_dict[name_stem].append(c.replace(name_stem, '').replace("_", ''))
df_pre_post_pair = df_interested[["randomid"] + pre_post_vars].copy().reset_index(drop = True)
df_pre_post_pair["pre_post_count"] = 0
for name_stem in pre_post_dict:
    if len(pre_post_dict[name_stem]) == 2:
        pre_index = df_pre_post_pair[df_pre_post_pair["pre_{}".format(name_stem)].notnull()].index
        post_index = df_pre_post_pair[df_pre_post_pair["post_{}".format(name_stem)].notnull()].index
        both_index = set(pre_index).intersection(set(post_index))
        df_pre_post_pair.loc[both_index, "pre_post_count"] += 1
df_pre_post_pair["pre_post_count"] /= df_pre_post_pair.shape[0]

## Mapping non-numerical column
for name_stem in pre_post_dict:
    if len(pre_post_dict[name_stem]) == 2:
        column_map = grade_scale if "grade" in name_stem else scale_map[name_stem] if name_stem in scale_map else None
        if "grade" in name_stem or name_stem in scale_map:
            df_pre_post_pair["pre_{}".format(name_stem)] = df_pre_post_pair["pre_{}".format(name_stem)].map(column_map)
            df_pre_post_pair["post_{}".format(name_stem)] = df_pre_post_pair["post_{}".format(name_stem)].map(column_map)

        
# df_pre_post_pair = df_pre_post_pair[df_pre_post_pair["pre_post_count"] >= 0.5]
df_pre_post_pair_describe = df_pre_post_pair.describe()

In [6]:
pre_post_inc, pre_post_dec = {}, {}
for name_stem in pre_post_dict:
    if len(pre_post_dict[name_stem]) == 2:
        if "pre_{}".format(name_stem) not in df_pre_post_pair_describe.columns or \
            "post_{}".format(name_stem) not in df_pre_post_pair_describe.columns:
            print(name_stem)
        else:
            pre = df_pre_post_pair_describe.loc["mean", "pre_{}".format(name_stem)]
            post = df_pre_post_pair_describe.loc["mean", "post_{}".format(name_stem)]
            if pre > post:
                pre_post_dec[name_stem] = post - pre
            else:
                pre_post_inc[name_stem] = post - pre

start
end
distribution


In [9]:
# Variables that decreased after class (post - pre < 0)
# temp = pd.DataFrame({"Name Stem": list(pre_post_dec.keys()), "Difference": list(pre_post_dec.values())})
# temp.merge(df_codebook["Pre & Post"][["Name Stem", "Item", "Label", "Response Values", "Construct"]],
#                                         left_on = "Name Stem", right_on = "Name Stem")

In [10]:
# Variables that increased after class (post - pre < 0)
# temp = pd.DataFrame({"Name Stem": list(pre_post_inc.keys()), "Difference": list(pre_post_inc.values())})
# temp.merge(df_codebook["Pre & Post"][["Name Stem", "Item", "Label", "Response Values", "Construct"]],
#                                         left_on = "Name Stem", right_on = "Name Stem")

In [7]:
pre_post_inc.update(pre_post_dec)
temp = pd.DataFrame({"Name Stem": list(pre_post_inc.keys()), "Post - Pre": list(pre_post_inc.values())})
temp.merge(df_codebook["Pre & Post"][["Name Stem", "Item", "Label", "Response Values", "Construct"]],
                                        left_on = "Name Stem", right_on = "Name Stem").sort_values("Construct")\
                                                                                      .reset_index(drop = True)\
                                                                                      .to_csv("../data/Pre_Post_Difference.csv", index = False)